# AutoGen

In [17]:
# pip install --upgrade pyautogen

In [1]:
import openai
import os
import getpass
from autogen import ConversableAgent

C:\Users\ttaul\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
# Set up the OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [6]:
llm_config = {"model": "gpt-4o-mini"}

In [8]:
# Define Alice, a friendly AI assistant agent
alice = ConversableAgent(
    name="alice",
    system_message="Your name is Alice, and you are a friendly AI assistant ready to help with any questions.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [9]:
# Define Bob, a curious learner agent
bob = ConversableAgent(
    name="bob",
    system_message="Your name is Bob, and you are a curious learner who loves asking questions.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [10]:
# Start a conversation where Bob asks Alice a question
chat_result = bob.initiate_chat(
    recipient=alice,
    message="Hi Alce! Can you tell me how photosynthesis works?",
    max_turns=2,
)


bob (to alice):

Hi Alice! Can you tell me how photosynthesis works?

--------------------------------------------------------------------------------
alice (to bob):

Hi Bob! Of course! Photosynthesis is the process by which green plants, algae, and certain bacteria convert light energy into chemical energy in the form of glucose. Here’s how it works:

1. **Light Absorption**: Photosynthesis primarily occurs in the chloroplasts of plant cells, where a green pigment called chlorophyll captures sunlight.

2. **Water and Carbon Dioxide**: Plants take in water (H₂O) from the soil through their roots and carbon dioxide (CO₂) from the air through small openings on their leaves called stomata.

3. **Chemical Reaction**: Using the energy absorbed from sunlight, the chlorophyll facilitates a chemical reaction that combines the water and carbon dioxide to produce glucose (C₆H₁₂O₆) and oxygen (O₂). The overall equation for photosynthesis can be summarized as:
   \[
   6 CO₂ + 6 H₂O + light \ ene

# Tweet Creator

In [29]:
llm_config = {"model": "gpt-4o-mini"}

In [33]:
task = '''
        Write an engaging tweet to promote a new AI tool designed for content creators. 
        The tweet should be concise, include relevant hashtags, and be within the character limit for Twitter.
       '''

In [3]:
import autogen

In [35]:
# Tweet Writer Agent
tweet_writer = autogen.AssistantAgent(
    name="TweetWriter",
    system_message="You are a tweet writer. You write concise and engaging tweets on given topics. "
                   "Your tweets should be compelling, include relevant hashtags, and be within the character limit.",
    llm_config=llm_config,
)

In [36]:

# Content Optimizer Agent
content_optimizer = autogen.AssistantAgent(
    name="ContentOptimizer",
    system_message="You are a content optimizer. You refine the tweet to improve its clarity, engagement, and impact. "
                   "Your revisions should enhance the message while keeping the tweet concise and within the character limit.",
    llm_config=llm_config,
)


In [37]:

# SEO Reviewer Agent
seo_reviewer = autogen.AssistantAgent(
    name="SEOReviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer. You optimize the tweet for search engines and social media algorithms. "
                   "Your suggestions should include relevant keywords and hashtags that increase visibility.",
)


In [38]:
# Legal Reviewer Agent
legal_reviewer = autogen.AssistantAgent(
    name="LegalReviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer. You ensure that the tweet is legally compliant and free from any potential legal issues. "
                   "Your review should be concise, ensuring that the content adheres to legal standards.",
)

In [39]:
# Final Reviewer Agent (finalizes the tweet)
final_reviewer = autogen.AssistantAgent(
    name="FinalReviewer",
    llm_config=llm_config,
    system_message="You are the final reviewer. You aggregate all feedback and finalize the tweet, ensuring it is optimized, legally compliant, and engaging.",
)

In [40]:
# Function to generate reflection messages
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

In [41]:

# Set up the nested review process
review_chats = [
    {
        "recipient": content_optimizer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt": 
            "Return review as a JSON object only:"
            "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
        "max_turns": 1
    },
    {
        "recipient": seo_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt": 
            "Return review as a JSON object only:"
            "{'Reviewer': '', 'Review': ''}.",},
        "max_turns": 1
    },
    {
        "recipient": legal_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt": 
            "Return review as a JSON object only:"
            "{'Reviewer': '', 'Review': ''}",},
        "max_turns": 1
    },
    {
        "recipient": final_reviewer,
        "message": "Aggregate feedback from all reviewers and finalize the tweet.",
        "max_turns": 1
    },
]


In [42]:
# Register the nested chats
tweet_writer.register_nested_chats(
    review_chats,
    trigger=tweet_writer,
)

In [43]:

# Initiate the chat and get the final tweet
res = tweet_writer.generate_reply(messages=[{"content": task, "role": "user"}])
res = final_reviewer.initiate_chat(
    recipient=tweet_writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

print(res.summary)



FinalReviewer (to TweetWriter):


        Write an engaging tweet to promote a new AI tool designed for content creators. 
        The tweet should be concise, include relevant hashtags, and be within the character limit for Twitter.
       

--------------------------------------------------------------------------------
TweetWriter (to FinalReviewer):

🚀 Unlock your creative potential with our new AI tool! 🎨✨ Streamline your content creation process and let your ideas shine. Say goodbye to writer’s block! 💡🔗 #ContentCreators #AI #CreativeTools #Innovation #WritersBlockBeGone

--------------------------------------------------------------------------------
FinalReviewer (to TweetWriter):

🚀 Unleash your creativity with our new AI tool designed just for content creators! 🎨✨ Say goodbye to writer’s block and elevate your workflow today! 💡🔗 #ContentCreators #AI #CreativeTools #Innovation #WritersBlockBeGone

--------------------------------------------------------------------------------

# HR

In [10]:
from typing import Annotated
llm_config = {"model": "gpt-4o-mini"}

In [11]:
# Example leave request to be processed
leave_request_text = """
Employee: John Doe
Department: IT
Leave Type: Annual Leave
Leave Dates: 10/01/2024 - 10/05/2024
Total Days: 5
Reason: Vacation
Remaining Leave Balance: 10 days
Status: Pending Approval
"""


In [ ]:
def summarize_leave_request() -> Annotated[str, "Summary of the leave request"]:
    """Summarizes the provided leave request."""
    return ("Summary: Employee John Doe from the IT department has requested 5 days "
            "of annual leave from 10/01/2024 to 10/05/2024 for vacation. "
            "Remaining leave balance is 10 days. Status is Pending Approval.")

def approve_or_reject_leave(summary: Annotated[str, "Summary of the leave request"]) -> Annotated[str, "Approval or Rejection decision"]:
    """Approves or rejects the leave request based on the summary."""
    if "Remaining leave balance is 10 days" in summary and "5 days" in summary:
        return "Approved: The leave request is approved as it meets the company policy."
    else:
        return "Rejected: The leave request is rejected due to insufficient leave balance."

In [12]:
# Leave Request Reviewer Agent
leave_request_reviewer = ConversableAgent(
    name="Leave Request Reviewer",
    system_message="You are responsible for reviewing the leave request. "
    "First, call summarize_leave_request() to get a summary of the leave request. "
    "Then, pass the summary to the Leave Approver for a decision.",
    llm_config=llm_config,
)

# Leave Approver Agent
leave_approver = ConversableAgent(
    name="Leave Approver",
    system_message="You are responsible for approving or rejecting the leave request. "
    "Wait for the summary from the Leave Request Reviewer and make a decision by calling approve_or_reject_leave(summary).",
    llm_config=llm_config,
)


In [13]:
# Registering tools with the agents
from autogen import register_function

# Register summarize_leave_request tool
register_function(
    summarize_leave_request,
    caller=leave_request_reviewer,
    executor=leave_request_reviewer,  # Set executor
    name="summarize_leave_request",
    description="Summarizes the leave request.",
)

# Register approve_or_reject_leave tool
register_function(
    approve_or_reject_leave,
    caller=leave_approver,
    executor=leave_approver,  # Set executor
    name="approve_or_reject_leave",
    description="Makes a decision to approve or reject the leave request based on the summary.",
)

In [14]:
# Manually call the functions to simulate the process
summary_result = summarize_leave_request()
decision_result = approve_or_reject_leave(summary_result)

# Output the results
print(f"Summary: {summary_result}")
print(f"Decision: {decision_result}")

Summary: Summary: Employee John Doe from the IT department has requested 5 days of annual leave from 10/01/2024 to 10/05/2024 for vacation. Remaining leave balance is 10 days. Status is Pending Approval.
Decision: Approved: The leave request is approved as it meets the company policy.
